The DetectTime_General notebook is design to calculate a detection time grid given a distribution of seismic stations.

The input file is assumed to be a list of latitude and longitudes with headers (LAT, LON)representing the csv columns. 
LAT,LON
19.1,-67.5
18.7,-70.3
...
Other columns may be present but are not employed for the calculation. Note for pandas the header name is case sensetive. 

The output file is a csv of the grid with event latitude (EVLA), event longitude (EVLO), Detection Time 1 (Time_4STNS), Detection time 2 (Time_8STNS). Time is in seconds and the location in decimal degrees. The CSV can be transfered and plotted in any plotting tool. A PyGMT notebook for plotting is included in the release as an example. 

The code requires Obspy, Pandas and Numpy

In [1]:
#import Libraries

import obspy
from obspy.taup import plot_travel_times
from obspy.taup import TauPyModel
from obspy.geodetics.base import gps2dist_azimuth
from obspy.geodetics.base import kilometers2degrees
from obspy.taup import plot_travel_times
import pandas as pd
import numpy as np


In [9]:
stnfile='carib_stats.csv'
st_df=pd.read_csv(stnfile)
print(st_df)

       N                           LOCATION        LAT         LON  \
0      1                  FLORIDA PENINSULA  27.036100  -80.361800   
1      2                  FLORIDA PENINSULA  25.865700  -80.907000   
2      3                      CENTRAL TEXAS  32.001500  -95.808400   
3      4                      CENTRAL TEXAS  30.782700  -97.585000   
4      5                          LOUISIANA  30.749800  -93.189800   
..   ...                                ...        ...         ...   
232  233                        MISSISSIPPI  32.218500  -90.517700   
233  234                         COSTA RICA  10.866400  -85.326000   
234  235                     CENTRAL MEXICO  22.769000 -102.567000   
235  236                 PUERTO RICO REGION  18.254189  -65.640482   
236  237  NEAR THE COAST OF GUERRERO MEXICO  17.606869 -101.464987   

           STN_CHAN  PERCENT_UP  
0    060A_HHZ.N4.00       100.5  
1    061Z_HHZ.N4.00       100.3  
2    237B_BHZ.TX.--        64.4  
3    435B_BHZ.TX.--    

In [5]:
#Choose velocity Model; custom models can be used if in Obspy format see Obspy manual for details
model = TauPyModel(model="IASP91")
#Latitude and longitude limits for the grid calculation. 
evlo_min=-90
evlo_max=-50
evla_min=-5
evla_max=35

#event depth,evdp, (here 10 km for shallow crustal events) and grid step size in degrees. 
evdp=10
step=0.1

#Output file for Travel times
ofile='Seismic_TT_test.csv'

#minimum stations for detection (Note python means that this is n-1)  ; Currently coded to get Traveltime to th 4th and 8th station
nstn=3
nstn2=7


In [7]:
evla=evla_min
df_out = pd.DataFrame(columns=('EVLO', 'EVLA', 'Time_4STNS','Time_8STNS'))
count=0

while evla <= evla_max:
    evlo=evlo_min
    while evlo <= evlo_max:
        tmpdist=[]
        for i in st_df.index:
            gomidist = kilometers2degrees((gps2dist_azimuth(st_df.LAT[i],st_df.LON[i],evla,evlo)[0]/1000))
            tmpdist.append(gomidist)
           
        dist_sort=np.sort(tmpdist)
#        print(dist_sort)
        arrivals_4stn=model.get_travel_times(source_depth_in_km=evdp, distance_in_degree=dist_sort[nstn], phase_list =['Pg','Pn','P','p','PKP','PP'])
        arrivals_8stn=model.get_travel_times(source_depth_in_km=evdp, distance_in_degree=dist_sort[nstn2], phase_list =['Pg','Pn','P','p','PKP','PP'])
        df_out.loc[count] =[evlo,evla,arrivals_4stn[0].time,arrivals_8stn[0].time]
        evlo=evlo+step
        count=count+1
    print(evlo,evla,arrivals_4stn[0].time,arrivals_8stn[0].time)
    evla=evla+step


-49.90000000000128 -5 279.3373967602105 289.46574435878176
-49.90000000000128 -4.9 278.4460444803625 288.5425125374634
-49.90000000000128 -4.800000000000001 277.5556396696706 287.61964905124296
-49.90000000000128 -4.700000000000001 276.666218489448 286.69718508218625
-49.90000000000128 -4.600000000000001 275.77781989744756 285.7751508671728
-49.90000000000128 -4.500000000000002 274.8904875514407 284.8535810822392
-49.90000000000128 -4.400000000000002 274.0042330227894 283.9325067280411
-49.90000000000128 -4.3000000000000025 273.11908150973767 283.01196092293753
-49.90000000000128 -4.200000000000003 272.23507236987354 282.0919779761231
-49.90000000000128 -4.100000000000003 271.3522412111445 281.1725949736099
-49.90000000000128 -4.0000000000000036 270.4706238759956 280.2538604705437
-49.90000000000128 -3.9000000000000035 269.5902548664298 279.3357832874471
-49.90000000000128 -3.8000000000000034 268.7111740440998 278.418378782151
-49.90000000000128 -3.7000000000000033 267.8334163584374 27

In [8]:
print(df_out)
df_out.to_csv(ofile)

        EVLO  EVLA  Time_4STNS  Time_8STNS
0      -90.0  -5.0  219.067117  225.335419
1      -89.9  -5.0  218.457139  224.931752
2      -89.8  -5.0  217.853376  224.535238
3      -89.7  -5.0  217.255900  224.145928
4      -89.6  -5.0  216.664782  223.697114
...      ...   ...         ...         ...
160395 -50.4  34.9  275.265845  280.074316
160396 -50.3  34.9  275.797997  280.610487
160397 -50.2  34.9  276.332889  281.134351
160398 -50.1  34.9  276.870484  281.656142
160399 -50.0  34.9  277.410754  282.180614

[160400 rows x 4 columns]
